In [1]:
import subprocess
import numpy as np
import pandas as pd
import scanpy as sc
import random
import string
import subprocess
import anndata as an
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy 
import warnings
from collections import Counter
from google.cloud import storage
import re
import pyarrow.feather as feather
from scipy import sparse
from anndata import AnnData

In [3]:
subprocess.run("gsutil cp gs://maus-cellranger/UMIcount_data.txt .", shell = True)

Copying gs://maus-cellranger/UMIcount_data.txt...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

/ [1 files][  3.2 GiB/  3.2 GiB]      0.0 B/s                                   
Operation completed over 1 objects/3.2 GiB.                                      


CompletedProcess(args='gsutil cp gs://maus-cellranger/UMIcount_data.txt .', returncode=0)

In [1]:
umi_count = pd.read_csv("UMIcount_data.txt", sep= "\t")
umi_t = umi_count.T
umi_clean = umi_t.iloc[1:,2:]

In [37]:
obs = pd.read_csv("/data/ccle_obs.csv")
obs.index = obs['index']
obs = obs.drop(columns = 'index')

In [7]:
all_cell = sc.read_h5ad("ccle_merge_cpm.h5ad")

In [66]:
new_name = [str.replace(n,'.','-') for n in all_cell.obs.Barcode]

In [91]:
new_name=[]
for n in all_cell.obs.Barcode:
    if n[0]=='X':
        n=n[1:]
    new_name.append(str.replace(n,'.','-'))

In [88]:
umi_name = []

for a,b in zip(umi_count.iloc[0,1:], umi_count.columns[1:]): 
    if len(re.findall( "UPPER_AERODIGESTIVE_TRACT",a))!=0:
        b = str.replace(b,'c','')
    if a[0]=='X':
        a = a[1:]
    umi_name.append(a+"-"+b)

In [105]:
match_location = [np.where(np.asarray(umi_name) == n)[0][0] for n in new_name]

In [106]:
len(match_location)

53513

In [107]:
match_umi = umi_clean.iloc[match_location,:]

In [116]:
match_umi.columns = umi_count.iloc[:,0][2:]

In [125]:
array = np.asarray(match_umi).astype(float)

In [126]:
sparse_X = sparse.csr_matrix(array)

In [127]:
from anndata import AnnData

In [129]:
umi_adata = AnnData(sparse_X)
umi_adata.obs = obs
umi_adata.var = pd.DataFrame(umi_count.iloc[:,0][2:])

In [130]:
umi_adata

AnnData object with n_obs × n_vars = 53513 × 30314
    obs: 'Barcode', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'CCLE_ID', 'index.1', 'cancer_type', 'pool_id', 'n_cells', 'cancer_type_trunc'
    var: 'Unnamed: 0'

In [132]:
umi_adata.write_h5ad("ccle_umi.h5ad")
subprocess.run("gsutil cp ccle_umi.h5ad gs://maus-cellranger/", shell = True)

Copying file://ccle_umi.h5ad [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

- [1 files][  1.5 GiB/  1.5 GiB]   83.7 MiB/s                                   
Operation completed over 1 objects/1.5 GiB.                                      


CompletedProcess(args='gsutil cp ccle_umi.h5ad gs://maus-cellranger/', returncode=0)

In [133]:
umi_adata.layers['umi'] = umi_adata.X

In [134]:
sc.pp.normalize_total(umi_adata, target_sum=1e4)
sc.pp.log1p(umi_adata)
sc.pp.highly_variable_genes(umi_adata, min_mean=0.01, max_mean=3, min_disp=0.2)

In [145]:
umi_adata.var.index = umi_adata.var['Unnamed: 0']
umi_adata.var.index.names=["index"]

In [146]:
umi_adata.var

,Unnamed: 0,highly_variable,means,dispersions,dispersions_norm
index,,,,,
A1BG,A1BG,True,0.226660,0.179397,0.896574
A1BG-AS1,A1BG-AS1,False,0.032659,-0.244404,-0.034471
A1CF,A1CF,False,0.007306,-0.058599,0.373721
A2M,A2M,True,0.035917,0.849223,2.368109
A2M-AS1,A2M-AS1,False,0.002309,-0.353224,-0.273538
...,...,...,...,...,...
SPATA13,SPATA13,False,0.068897,-0.154249,0.163589
TBC1D26,TBC1D26,False,0.000039,-0.621727,-0.863410
TIMM10B,TIMM10B,False,0.262999,-0.227587,-0.577479


In [147]:
umi_adata.var.loc[['XIST','RPS4Y1','RPS4Y2'],'highly_variable'] = False

In [148]:
umi_adata.var[umi_adata.var['Unnamed: 0'].isin(['XIST','RPS4Y1','RPS4Y2'])]

,Unnamed: 0,highly_variable,means,dispersions,dispersions_norm
index,,,,,
RPS4Y1,RPS4Y1,False,0.869183,2.230466,3.572005
RPS4Y2,RPS4Y2,False,0.001376,-0.693808,-1.021764
XIST,XIST,False,0.231898,1.067043,2.846636


In [149]:
np.sum(umi_adata.var.highly_variable)

3852

In [150]:
umi_adata.raw = umi_adata
umi_adata = umi_adata[:,umi_adata.var['highly_variable']]

In [154]:
Counter(umi_adata.obs['CCLE_ID'])

Counter({'NCIH2126_LUNG': 264,
         'SW579_THYROID': 217,
         'C32_SKIN': 554,
         'NCIH446_LUNG': 196,
         'HEC251_ENDOMETRIUM': 107,
         'MFE319_ENDOMETRIUM': 191,
         'SKNAS_AUTONOMIC_GANGLIA': 226,
         'NCIH2452_PLEURA': 538,
         'COLO741_SKIN': 329,
         'WM88_SKIN': 242,
         'JHH7_LIVER': 198,
         'KNS42_CENTRAL_NERVOUS_SYSTEM': 172,
         'MCF7_BREAST': 143,
         'HT1197_URINARY_TRACT': 74,
         'SNU899_UPPER_AERODIGESTIVE_TRACT': 83,
         'HCC38_BREAST': 200,
         'HEC108_ENDOMETRIUM': 172,
         'HT1376_URINARY_TRACT': 120,
         'SNU308_BILIARY_TRACT': 94,
         'TYKNU_OVARY': 117,
         'SW1271_LUNG': 115,
         'TM31_CENTRAL_NERVOUS_SYSTEM': 116,
         'NCIH747_LARGE_INTESTINE': 80,
         'KPNSI9S_AUTONOMIC_GANGLIA': 129,
         'SQ1_LUNG': 238,
         'CAOV3_OVARY': 221,
         'IALM_LUNG': 322,
         'BT474_BREAST': 125,
         'DKMG_CENTRAL_NERVOUS_SYSTEM': 271,
      

In [170]:
umi_adata.write_h5ad("ccle_umi_process.h5ad")
subprocess.run("gsutil cp ccle_umi_process.h5ad gs://maus-cellranger/", shell = True)

Copying file://ccle_umi_process.h5ad [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\ [1 files][  2.2 GiB/  2.2 GiB]   90.2 MiB/s                                   
Operation completed over 1 objects/2.2 GiB.                                      


CompletedProcess(args='gsutil cp ccle_umi_process.h5ad gs://maus-cellranger/', returncode=0)

### NMF by cell lines

In [155]:
import os

In [156]:
os.getcwd()

'/home/jupyter/Getz_lab'

In [158]:
os.chdir('/home/jupyter/SignatureAnalyzer-GPU/')

In [159]:
os.getcwd()

'/home/jupyter/SignatureAnalyzer-GPU'

In [164]:
umi_adata.obs.cancer_type

index
NCIH2126_LUNG.AAACCTGAGACATAAC.1.18               Lung Cancer
NCIH2126_LUNG.AACGTTGTCACCCGAG.1.18               Lung Cancer
NCIH2126_LUNG.AACTGGTAGACACGAC.1.18               Lung Cancer
NCIH2126_LUNG.AACTGGTAGGGCTTGA.1.18               Lung Cancer
NCIH2126_LUNG.AACTGGTAGTACTTGC.1.18               Lung Cancer
                                                 ...         
JHU006_UPPER_AERODIGESTIVE_TRACT.4722    Head and Neck Cancer
JHU006_UPPER_AERODIGESTIVE_TRACT.4724    Head and Neck Cancer
JHU006_UPPER_AERODIGESTIVE_TRACT.4731    Head and Neck Cancer
JHU006_UPPER_AERODIGESTIVE_TRACT.4735    Head and Neck Cancer
JHU006_UPPER_AERODIGESTIVE_TRACT.4741    Head and Neck Cancer
Name: cancer_type, Length: 53513, dtype: category
Categories (22, object): ['Bile Duct Cancer', 'Bladder Cancer', 'Bone Cancer', 'Brain Cancer', ..., 'Prostate Cancer', 'Sarcoma', 'Skin Cancer', 'Thyroid Cancer']

In [169]:
for c in umi_adata.obs['cancer_type'].unique()[3:]:
    
    print(c)
    patient = umi_adata[umi_adata.obs['cancer_type'] == c]
    
    X = pd.DataFrame(patient.layers['umi'].todense().T,
                     index=list(patient.var.index),
                     columns=list(patient.obs.index))
    
    name = str.replace(c," ","_")
    name = str.replace(name, "/","_")
    feather.write_feather(X, name)
    
    command = f"python3 SignatureAnalyzer-GPU.py --data {name} --feather --prior_on_W L1 --prior_on_H L2 --output_dir ~/{name} --parameters_file ~/Getz_lab/NMF_analysis/default_phi1_iter5.txt --max_iter 20000 --labeled --tolerance 1e-5 --force_no_val_set"
    
    subprocess.run(command, shell = True)

Endometrial/Uterine Cancer
Reading data frame from Endometrial_Uterine_Cancer
loading feather...
NMF class initalized.
/home/jupyter/Endometrial_Uterine_Cancer
NMF data and parameters set.
   * Using GPU: cuda:0
%%%%%%%%%%%%%%%
a = 10
b = 0.00012778398959310794
%%%%%%%%%%%%%%%
nit=    0 K=   40 | obj=683906.38	b_div=1065989.38	lam=2.07	del=0.99999303	sumW=9870.54	sumH=5291.85
nit=  100 K=   12 | obj=-1529073.50	b_div=805751.25	lam=1.01	del=0.00461976	sumW=3179.04	sumH=4641.00
nit=  200 K=   12 | obj=-1539309.62	b_div=795585.44	lam=1.06	del=0.00745139	sumW=3423.00	sumH=4199.65
nit=  300 K=   11 | obj=-1603895.25	b_div=802002.94	lam=1.04	del=0.00297455	sumW=3390.19	sumH=3798.31
nit=  400 K=   10 | obj=-1663934.62	b_div=807239.38	lam=1.04	del=0.05836526	sumW=3376.56	sumH=3526.84
nit=  500 K=   10 | obj=-1665524.25	b_div=806120.88	lam=1.04	del=0.00218731	sumW=3368.95	sumH=3511.66
nit=  600 K=   10 | obj=-1666545.88	b_div=808138.75	lam=1.04	del=0.02798382	sumW=3361.48	sumH=3446.04
nit=  700

TypeError: descriptor 'replace' requires a 'str' object but received a 'float'

python3 SignatureAnalyzer-GPU.py --data 'Lung Cancer' --feather --prior_on_W L1 --prior_on_H L2 --output_dir ~/Lung_ Cancer --parameters_file ~/Getz_lab/NMF_analysis/default_phi1_iter5.txt --max_iter 20000 --labeled --tolerance 1e-5 --force_no_val_set

python3 SignatureAnalyzer-GPU.py --data NCIH2126_LUNG --feather --prior_on_W L1 --prior_on_H L2 --output_dir ~/NCIH2126_LUNG --parameters_file ~/Getz_lab/NMF_analysis/default_phi1.txt --max_iter 20000 --labeled --tolerance 1e-5 --force_no_val_set
    